In [69]:
import tensorflow as tf
import gym
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, ReLU
from tensorflow.keras.optimizers import Adam
import time
import scipy.signal
from tqdm import tqdm

# TrajectoryStorage


In [70]:
class Storage:
    def __init__(self):
        # init creating a storage for all observation variables during trajectory
        self.observations = []
        # create arrays for chosen actions and rewards
        self.actions = []
        self.logits = []
        self.rewards = []
        self.BaselineEstimate = []
        # finished episodes will be completely stored in this list 
        self.episodes = []


    def store(self,observation, action, logits, reward, BaselineEstimate):
        self.observations.append(observation)
        self.actions.append(action)
        self.logits.append(logits)
        self.rewards.append(reward)
        self.BaselineEstimate.append(BaselineEstimate) # value of critics network
        

    def conclude_episode(self):
        # append all already stored values to finished episodes
        self.episodes.append(
            [self.observations,
             self.actions, 
             self.logits,
             self.rewards,
             self.BaselineEstimate, 
             sum(self.rewards)]) # get the return of the whole episode
             
        # empty the arrays for new trajectory
        self.observations.clear()
        # create arrays for chosen actions and rewards
        self.actions.clear()
        self.logits.clear()
        self.rewards.clear()
        self.BaselineEstimate.clear()


    # return array of finished trajectories stored in self.episodes and the amount of episodes
    def get_episodes(self):
        return self.episodes, len(self.episodes)
        
        

# Actor Model


In [71]:
class Actor(Model):
    def __init__(self):
        super(Actor, self).__init__()


        self.l = [
            Dense(128, activation="relu", kernel_initializer=tf.random_normal_initializer
            (stddev=0.01)),
            Dense(128, activation="relu", kernel_initializer=tf.random_normal_initializer
            (stddev=0.01)),
            Dense(64, activation="relu", kernel_initializer=tf.random_normal_initializer
            (stddev=0.01)),
            Dense(4, activation="softmax")
        ]

    #@tf.function        
    def call(self, x):
        for l in self.l:
            x = l(x)
        return x

        

#@tf.function
def sample_action(observation):
    logits = actor(observation)
   # tf.print(type(logits))
    action = tf.squeeze(tf.random.categorical(logits, 1), axis=1)
   # tf.print(action)
    return logits, action

# Critic Model

In [72]:
class Critic(Model):
    def __init__(self):
        super(Critic, self).__init__()

        self.l = [
            Dense(128, activation="relu", kernel_initializer=tf.random_normal_initializer
            (stddev=0.01)),
            Dense(128, activation="relu", kernel_initializer=tf.random_normal_initializer
            (stddev=0.01)),
            Dense(64, activation="relu", kernel_initializer=tf.random_normal_initializer
            (stddev=0.01)),
            Dense(1, activation="tanh", kernel_regularizer=tf.random_normal_initializer(stddev=0.01))
        ]

    #@tf.function 
    def call(self, x):
        for l in self.l:
            x = l(x)
        return x

In [73]:
# define Hyperparameters
epochs = 1
steps_per_epoch = 1000 # ~10 Episodes per epoch, then compute new parameters (smaller batching)
lr_actor = 3e-4
lr_critic = 3e-4
train_policy_iterations = 80
train_value_iterations = 80
clip_ratio = 0.2
target_kl = 0.01
optimizer = Adam()

render = False

In [74]:
tf.keras.backend.clear_session()

# define environment
env = gym.make("LunarLander-v2")
# get observation_dims and amount of possible actions (1 for CartPole-v1)
observation_dimensions = env.observation_space.shape[0]
num_actions = env.action_space.n

# create Storage for observations, actions, rewards etc during trajectory
T = Storage()

# init the actor and critics model
observation_input = Input(shape=(observation_dimensions,), dtype=tf.float32)
actor = Actor()
critic = Critic()

# Initialize the observation, episode return and episode length
observation, episode_return, episode_length = env.reset(), 0, 0

In [75]:
episodes_total = 0
for epoch in range(epochs):

    sum_return = 0
    sum_length = 0
    num_episodes = 0

    for t in tqdm(range(steps_per_epoch)):
        if render:
            env.render()

        observation = observation.reshape(1,-1)

        # obtain action and logits for this observation by our actor
        logits, action = sample_action(observation=observation)
        
        # make a step in environment and obtain the rewards for it
        observation_new, reward, done, _ = env.step(action[0].numpy())

        # sum up rewards over this episode and count amount of frames
        episode_return += reward
        episode_length += 1

        # get the Base-Estimate from the Critics network
        base_estimate = critic(observation)

        # store Variables collected in this step
        T.store(observation=observation, action=action, logits=logits, reward=reward, BaselineEstimate=base_estimate)
        #update the observations
        observation = observation_new
        # check if terminal state is reached in env, if so save episode and refresh storage, reset env
        if done:
            T.conclude_episode()
            observation, episode_return, episode_length = env.reset(), 0, 0

    # obtain all episodes saved in storage
    episodes, amount_episodes = T.get_episodes()

  

        


100%|██████████| 1000/1000 [00:03<00:00, 294.49it/s]


In [76]:
  # episodes [episode][particular values // 0: Observations, 1: actions, 2: logits, 3, rewards, 4: BaselineEstimates from Critics]
  #print(episodes[0][4])
  print(f'Number of Episodes = {amount_episodes}')



  ### Advantagefunction

  # estimated Value of the current situtation from the critics network
  b_estimates = episodes[0][4] 
  # for i in b_estimates:
  #   print(i.numpy())

  # Discounted sum of rewards
  print(episodes[0][3]) 
  rewards = episodes[0][3]
  gamma = 0.99

  # for r in rewards:
    



Number of Episodes = 10
[0.24739931818194918, 0.06340160536021358, -0.6875144136568281, 0.08992597747698142, 0.17760116758984282, 0.5746170633385941, 0.512447925666039, 1.809780911407256, 0.9084949772032302, -1.0988825563743194, -1.186213334398501, -1.8731402508958854, -0.9921908961385271, 2.326015483628635, -0.6560151809752028, 0.1882939850001162, 0.8843979878682944, 0.30247995545125833, -0.05557446739512101, 2.2209883540771616, 2.433475674100987, -0.44601470727846504, -1.6009836532969064, 0.43654815469460684, -1.4876729076680977, -0.7444631801743924, -1.577498300881074, -1.9451985987056435, 0.8261580559114747, -2.0189766902066695, -2.0078647561900014, 0.8916838217678105, -0.02630299810754194, -1.4520927638871228, -0.312115426449991, -0.42978526738465805, -1.092875183404601, -1.9099133868513423, 1.5606008506394005, -2.140308912948625, -0.36334049143965674, -1.2080496002411962, 1.176609399613153, -2.3566675627997413, -1.4835418834327356, -2.5763313293430756, -1.6738215089355606, 0.0245

In [90]:
def discounted_reward(rewards, gamma):
    i = 0
    discounted_rewards = []
    for r in rewards:
        disc = 0
        for t in rewards[i:-1]:
            discount_t = gamma ** t
            disc += t * discount_t
        i += 1
        discounted_rewards.append(disc)
    return discounted_rewards

        

In [91]:
disc_sum = discounted_reward(rewards, gamma)
#print(len(rewards))
#print(len(disc_sum))
print(disc_sum)

[-18.639445860826168, -18.886230798033385, -18.949592016288594, -18.257310598951452, -18.34715533928021, -18.524439780175445, -19.09574793950016, -19.605563402830835, -21.38272394315445, -22.282961499066484, -21.171875466864787, -19.971435649592774, -18.062698224243174, -17.060563852557955, -19.33283417567274, -18.672479483333905, -18.860417474398222, -19.73698932639326, -20.038551131482432, -19.98294561473931, -22.154906997774976, -24.52958845725035, -24.081569957772043, -22.454617434844277, -22.889254449492373, -21.379171316167163, -20.629117094018067, -19.02640921023492, -17.042808014624818, -17.86213474318333, -15.801771728773174, -13.752977247595512, -14.636705746880617, -14.610395794552085, -13.136955767568823, -12.823859740002556, -12.39221400581347, -11.287268774878576, -9.34033995895218, -10.876654426258668, -8.689806959439164, -8.325137234218138, -7.102330934767254, -8.265108496282895, -5.851956266524225, -4.346128907474705, -1.7022174938695223, 0]
